#Run cross-group analyses that lyman doesn't suppport

In [2]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
import scipy.stats

##nipype
import nibabel as nib

%matplotlib inline

In [3]:
#preliminary housekeeping
home_dir = '/data/home/iballard/mag/'
subj_file = home_dir + 'subjects.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)
exps = ['mag']
runs = map(str,range(1,5))

In [13]:
group_dir = home_dir + 'analysis/mag/randomise/'
if not os.path.exists(group_dir):
    os.mkdir(group_dir)

#Make design file

In [51]:
#alternatively, make correlation design file
# cov = np.loadtxt(home_dir + '/analysis/omegas_powell.txt')
# cov = np.loadtxt(home_dir + 'vta_mb.txt')
# cov = [x for x in cov if x> -5000]
cov = pd.read_csv(home_dir + 'ser_r2_diff.csv')
cov  = cov['diff_norm'].values 
cov = cov - np.mean(cov)
print cov
##make regressor and contrast inputs according to FSL rules for paired ttest
# regressors = dict(cov = list(cov) )
regressors = dict(cov = list(cov))
contrasts=[['ser-corr','T',['cov'],[1]]]
# regressors = dict(mean = list(np.ones(len(cov))))
# contrasts=[['ser-corr','T',['mean'],[1]]]

##set up WF so I can control where outputs go
model = Node(fsl.MultipleRegressDesign(regressors = regressors, contrasts = contrasts),name='model')
wf = Workflow(name = 'design',base_dir = group_dir)
sink = Node(DataSink(),name = 'sinker',base_directory = group_dir)
wf.connect(model,'design_con',sink,'output.@con')
wf.connect(model,'design_grp',sink,'output.@grp')
wf.connect(model,'design_mat',sink,'output.@mat')
wf.run()

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node model in dir: /data/home/iballard/fd/analysis/group_ser_sim_4mm_PEmb/design/model
INFO:workflow:Collecting precomputed outputs
INFO:workflow:Executing node sinker in dir: /data/home/iballard/fd/analysis/group_ser_sim_4mm_PEmb/design/sinker


[ 0.02670384  0.0166462   0.01692453 -0.00308281 -0.00612884 -0.01236596
  0.03474843  0.03591418  0.00109689 -0.02691963 -0.00103481  0.00214077
  0.03820496 -0.00154382 -0.01608614  0.01880507 -0.04014717 -0.00477828
 -0.03147572 -0.03426227 -0.03910758 -0.04015113  0.03599758  0.00682268
 -0.00073862  0.01846181  0.01704517 -0.0092812   0.02781854 -0.04014885
 -0.00724877  0.017171  ]


# Merge files

In [31]:
#merge the 3d files from ffx analysis into concatenated 4d files
images = ['cope']
ims_to_files = {'cope':'cope1.nii.gz'}
contrast = 'high_minus_low'

for f in images:
    out_dir = group_dir + contrast
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    
    #add files to merge command
    out_f = out_dir + '/' + f + '_merged.nii.gz'
    os.remove(out_f)
    if not os.path.exists(out_f):
        cmd_str = ['fslmerge','-t',out_f]

        for exp in ['mag']: #order matters now
            for sub in sub_list:
                sub_dir = home_dir + '/analysis/' + exp + '/' + sub 
                sub_f = sub_dir + '/reg/mni/smoothed/high_minus_low.nii.gz'
                
#                 sub_f = sub_dir + '/ffx/mni/smoothed/' + contrast + '/' + ims_to_files[f]
                cmd_str.append(sub_f)
        cmd_str = ' '.join(cmd_str)
        os.system(cmd_str)

In [29]:
for exp in ['mag']: #order matters now
    for sub in sub_list:
        sub_dir = home_dir + '/analysis/' + exp + '/' + sub + '/reg/mni/smoothed/'
        f1 = sub_dir + '/run_1/cope25_warp.nii.gz' 
        f2 = sub_dir + '/run_2/cope25_warp.nii.gz'
        out1 = sub_dir + 'low.nii.gz'
        
        cmd = ['fslmaths',f1,'-add',f2,out1]
        cmd = ' '.join(cmd)
        os.system(cmd)
        
        f1 = sub_dir + '/run_3/cope25_warp.nii.gz' 
        f2 = sub_dir + '/run_4/cope25_warp.nii.gz'
        out2 = sub_dir + 'high.nii.gz'
        
        cmd = ['fslmaths',f1,'-add',f2,out2]
        cmd = ' '.join(cmd)      
        os.system(cmd)

        out3 = sub_dir + 'high_minus_low'
        cmd = ['fslmaths',out1,'-add',out2,out3]
        cmd = ' '.join(cmd) 
        os.system(cmd)
    

# Do fitting with randomise

In [21]:
im_dir = group_dir + contrast
stat_file = im_dir + '/cope_merged.nii.gz'
mask_file = home_dir + '/analysis/' + exp + '/group/mni/' + contrast + '/mask.nii.gz'

cmd_str = ['randomise','-i',stat_file,'-o',im_dir,'-m',mask_file,'-T','-v','5','-1','-n','250']
cmd_str = ' '.join(cmd_str)
os.system(cmd_str)
# print cmd_str

0

In [52]:
#do fitting with randomise
#input image files
def randomise(in_tuple):
    contrast,exp,altmodel = in_tuple

    im_dir = group_dir + contrast + '/'
    if not os.path.exists(im_dir):
        os.mkdir(im_dir)
        
#     stat_file = im_dir + 'cope_' + exp + '_merged.nii.gz'
#     mask_file = home_dir + '/analysis/ser_4mm-' + altmodel + '/group/mni/' + contrast + '/mask.nii.gz'
#     cmd = ['randomise','-i',stat_file,'-o',im_dir,'-m', mask_file,'-T','-1','-n','1000']
#     cmd = ' '.join(cmd)
#     os.system(cmd)

    #input design files
    stat_file = im_dir + '/corr/' + 'cope_' + exp + '_merged.nii.gz'
    out_dir = im_dir + '/corr/'
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    mask_file = home_dir + '/analysis/ser_4mm-' + altmodel + '/group/mni/' + contrast + '/mask.nii.gz'
    design_dir = group_dir + '/design/sinker/output/'
    t_con_file = design_dir + 'design.con'
    cov_split_file = design_dir + 'design.grp'
    design_file = design_dir + 'design.mat'
    cmd_str = ['randomise','-i',stat_file,'-o',out_dir,'-d',design_file,
               '-t',t_con_file,'-m',mask_file,'-n','1000','-D','-T']
    cmd_str = ' '.join(cmd_str)
    os.system(cmd_str)
    

In [53]:
in_tuples = [('PE_mb','ser','PEfb-diff'),('PE_mb_state','ser','PEfb-diff'),
            ('state','ser','PEfb-diff')]
pool = multiprocessing.Pool(processes = len(in_tuples))
pool.map(randomise,in_tuples)
pool.terminate()
pool.join()